## Feature Selection ##

In [13]:
import pandas as pd
import ta
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from pandas.plotting import scatter_matrix
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import RobustScaler

In [14]:
# Import data 
df = pd.read_csv('../raw_data/clean_raw_data.csv')
df.head()

,timestamp,open,high,low,close,volume,CMF,FII,MFI,VWA,...,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,2019-09-17 04:00:00,10216.35,10238.95,10194.39,10209.45,872.764,-0.120924,-913.266701,28.652105,10185.127432,...,10275.790895,3005.7,2.25,1.7,1,0.45992,0.46011,-1,48.386232,-1
1,2019-09-17 05:00:00,10208.81,10232.07,10204.53,10223.97,819.923,-0.114688,917.954536,36.501645,10187.270366,...,10274.443495,3005.7,2.25,1.7,-1,0.40934,0.45992,-1,42.873022,-1
2,2019-09-17 06:00:00,10224.11,10230.44,10187.88,10224.09,922.804,-0.112046,802.637671,36.306243,10191.939033,...,10273.130448,3005.7,2.25,1.7,-1,0.31251,0.40934,-1,42.291553,-1
3,2019-09-17 07:00:00,10225.02,10230.79,10192.69,10193.85,905.598,-0.124361,-3224.208213,35.631136,10195.919801,...,10271.256418,3005.7,2.25,1.7,-1,0.14629,0.31251,-1,41.595938,-1
4,2019-09-17 08:00:00,10193.21,10215.07,10165.00,10195.82,1089.829,-0.114965,-2456.898021,34.774165,10199.535893,...,10269.459713,3005.7,2.25,1.7,-1,-0.27719,0.14629,-1,41.970769,-1


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32556 entries, 0 to 32555
Data columns (total 78 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   timestamp               32556 non-null  object 
 1   open                    32556 non-null  float64
 2   high                    32556 non-null  float64
 3   low                     32556 non-null  float64
 4   close                   32556 non-null  float64
 5   volume                  32556 non-null  float64
 6   CMF                     32556 non-null  float64
 7   FII                     32556 non-null  float64
 8   MFI                     32556 non-null  float64
 9   VWA                     32556 non-null  float64
 10  AOI                     32556 non-null  float64
 11  KAMA                    32556 non-null  float64
 12  PPO                     32556 non-null  float64
 13  PPO_signal              32556 non-null  float64
 14  PVO                     32556 non-null

In [20]:
train_ratio = 0.8

In [24]:
# splitting the data into test and train
split_index = int(len(df) * train_ratio)


train_data = df[:split_index]
test_data = df[split_index:]

In [22]:
scaler = RobustScaler()

In [30]:
X_train = train_data.drop(columns=["close", "timestamp"])

In [33]:
X_scaled = scaler.fit_transform(X_train)
X_scaled

array([[-0.40945076, -0.40946536, -0.40851088, ..., -1.        ,
        -0.21117935, -1.        ],
       [-0.40967576, -0.40966932, -0.40820653, ..., -1.        ,
        -0.61729698, -1.        ],
       [-0.40921921, -0.40971764, -0.40870627, ..., -1.        ,
        -0.66012953, -1.        ],
       ...,
       [-0.12553484, -0.12338292, -0.12279363, ...,  0.        ,
         0.33101098, -1.        ],
       [-0.12244045, -0.12175539, -0.11937799, ...,  0.        ,
         0.37645873, -1.        ],
       [-0.12204059, -0.12292638, -0.11975317, ...,  0.        ,
         0.44722927, -1.        ]])

In [39]:
X_scaled=pd.DataFrame(X_scaled, columns=scaler.get_feature_names_out())

In [42]:
pd.concat([X_scaled, train_data[["timestamp"]]], axis=1)

,open,high,low,volume,CMF,FII,MFI,VWA,AOI,KAMA,...,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity,timestamp
0,-0.409451,-0.409465,-0.408511,-0.760314,-0.903963,0.004708,-0.794315,-0.410297,-0.030276,-0.411331,...,-0.792272,2.666667,-0.160714,1.0,0.163900,0.163986,-1.0,-0.211179,-1.0,2019-09-17 04:00:00
1,-0.409676,-0.409669,-0.408207,-0.764729,-0.865611,0.006897,-0.494177,-0.410232,-0.047055,-0.411261,...,-0.792272,2.666667,-0.160714,0.0,0.141062,0.163900,-1.0,-0.617297,-1.0,2019-09-17 05:00:00
2,-0.409219,-0.409718,-0.408706,-0.756133,-0.849362,0.006760,-0.501648,-0.410093,-0.080557,-0.411196,...,-0.792272,2.666667,-0.160714,0.0,0.097342,0.141062,-1.0,-0.660130,-1.0,2019-09-17 06:00:00
3,-0.409192,-0.409707,-0.408562,-0.757571,-0.925105,0.001945,-0.527462,-0.409974,-0.097027,-0.411189,...,-0.792272,2.666667,-0.160714,0.0,0.022291,0.097342,-1.0,-0.711370,-1.0,2019-09-17 07:00:00
4,-0.410141,-0.410173,-0.409393,-0.742177,-0.867313,0.002862,-0.560229,-0.409866,-0.103385,-0.411180,...,-0.792272,2.666667,-0.160714,0.0,-0.168916,0.022291,-1.0,-0.683759,-1.0,2019-09-17 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26039,-0.126672,-0.127536,-0.124420,0.604395,-0.534858,-0.393501,-0.553531,-0.123006,-0.189850,-0.123653,...,0.056455,3.000000,1.000000,1.0,-1.196678,-1.302201,-1.0,-0.076637,-1.0,2022-09-05 15:00:00
26040,-0.125985,-0.126558,-0.123145,0.554581,-0.201898,-0.293617,-0.680724,-0.123649,-0.190394,-0.123820,...,0.056455,3.000000,1.000000,1.0,-0.819283,-1.196678,-1.0,-0.301288,-1.0,2022-09-05 16:00:00
26041,-0.125535,-0.123383,-0.122794,1.049994,-0.072507,0.148010,-0.309472,-0.124046,-0.171083,-0.123807,...,0.056455,3.000000,1.000000,1.0,-0.407854,-0.819283,0.0,0.331011,-1.0,2022-09-05 17:00:00
26042,-0.122440,-0.121755,-0.119378,3.253399,-0.640404,0.240479,0.202481,-0.123648,-0.096253,-0.123695,...,0.056455,3.000000,1.000000,1.0,-0.052990,-0.407854,0.0,0.376459,-1.0,2022-09-05 18:00:00


In [60]:
X_scaled

,open,high,low,volume,CMF,FII,MFI,VWA,AOI,KAMA,...,wma_100,S&P_close_filled,Interest_Rates_filled,Inflation_Rate_us,subhag_signal,Fisher,Fisher_trigger,supertrend,chop_index,Velocity
0,-0.409451,-0.409465,-0.408511,-0.760314,-0.903963,0.004708,-0.794315,-0.410297,-0.030276,-0.411331,...,-0.405627,-0.792272,2.666667,-0.160714,1.0,0.163900,0.163986,-1.0,-0.211179,-1.0
1,-0.409676,-0.409669,-0.408207,-0.764729,-0.865611,0.006897,-0.494177,-0.410232,-0.047055,-0.411261,...,-0.405667,-0.792272,2.666667,-0.160714,0.0,0.141062,0.163900,-1.0,-0.617297,-1.0
2,-0.409219,-0.409718,-0.408706,-0.756133,-0.849362,0.006760,-0.501648,-0.410093,-0.080557,-0.411196,...,-0.405706,-0.792272,2.666667,-0.160714,0.0,0.097342,0.141062,-1.0,-0.660130,-1.0
3,-0.409192,-0.409707,-0.408562,-0.757571,-0.925105,0.001945,-0.527462,-0.409974,-0.097027,-0.411189,...,-0.405762,-0.792272,2.666667,-0.160714,0.0,0.022291,0.097342,-1.0,-0.711370,-1.0
4,-0.410141,-0.410173,-0.409393,-0.742177,-0.867313,0.002862,-0.560229,-0.409866,-0.103385,-0.411180,...,-0.405816,-0.792272,2.666667,-0.160714,0.0,-0.168916,0.022291,-1.0,-0.683759,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26039,-0.126672,-0.127536,-0.124420,0.604395,-0.534858,-0.393501,-0.553531,-0.123006,-0.189850,-0.123653,...,-0.119510,0.056455,3.000000,1.000000,1.0,-1.196678,-1.302201,-1.0,-0.076637,-1.0
26040,-0.125985,-0.126558,-0.123145,0.554581,-0.201898,-0.293617,-0.680724,-0.123649,-0.190394,-0.123820,...,-0.119604,0.056455,3.000000,1.000000,1.0,-0.819283,-1.196678,-1.0,-0.301288,-1.0
26041,-0.125535,-0.123383,-0.122794,1.049994,-0.072507,0.148010,-0.309472,-0.124046,-0.171083,-0.123807,...,-0.119634,0.056455,3.000000,1.000000,1.0,-0.407854,-0.819283,0.0,0.331011,-1.0
26042,-0.122440,-0.121755,-0.119378,3.253399,-0.640404,0.240479,0.202481,-0.123648,-0.096253,-0.123695,...,-0.119656,0.056455,3.000000,1.000000,1.0,-0.052990,-0.407854,0.0,0.376459,-1.0


In [43]:
## VIF Analysis
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [44]:
# compute VIF factor for feature index 0
vif(X_scaled.values, 0)

25485.658657052983

In [50]:
df=pd.DataFrame()

df["features"] = X_scaled.columns

df["vif_index"] = [vif(X_scaled.values, i) for i in range(X_scaled.shape[1])]

round(df.sort_values(by="vif_index", ascending = False),2)

/home/mehmet/.pyenv/versions/assist_trader/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,features,vif_index
38,ARI_up,inf
54,SMA_20,inf
25,BOL_avg,inf
24,BOL_hband,inf
23,BOL_lband,inf
...,...,...
70,subhag_signal,2.59
74,chop_index,2.56
73,supertrend,2.38
68,Interest_Rates_filled,2.18


In [74]:
## Features we can rely on
df.query("vif_index < 10").features

3                    volume
4                       CMF
5                       FII
6                       MFI
12                      PVO
13               PVO_signal
16                stoch_RSI
20       UltimateOscillator
33                      ADX
34                  ADX_neg
40                      DPO
51                      MAI
58                      STC
67         S&P_close_filled
68    Interest_Rates_filled
69        Inflation_Rate_us
70            subhag_signal
73               supertrend
74               chop_index
75                 Velocity
Name: features, dtype: object

In [73]:
df.features.unique().shape

(76,)

In [67]:
X_scaled= pd.concat([X_scaled, train_data[["close"]]],axis=1)

In [71]:
# Compute correlation matrix
corr_matrix = X_scaled.corr()

In [72]:
close_corr= corr_matrix["close"].sort_values(ascending=False)

In [11]:
# Visualize the Corr matrix
plt.figure(figsize=(100, 100))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()

--> The heatmap visualization will display the correlation coefficients between each pair of features in your dataset. 
The colors in the heatmap represent the strength and direction of the correlation. 
Positive correlations are indicated by warmer colors (reds), 
negative correlations by cooler colors (blues), and no correlation by white.


In [ ]:
# Compute autocorrelation
plot_acf(data['close'], lags=14)
plt.title("Autocorrelation Function")
plt.show()

In [ ]:
# Compute Partial Autocorrelation 
plot_pacf(data['close'], lags=14)
plt.title("Partial Autocorrelation Function")
plt.show()

In [ ]:
# Compute scatter matrix
subset = data["close"]

In [ ]:
scatter_matrix(subset, figsize=(10, 10), diagonal='kde')
plt.show()

### Granger Causality ###

In [ ]:
# put the feature you wanna analyze into subset
subset = data[["close", "volume"]]

In [ ]:
max_lag = 14  # Specify the maximum lag value for the test
results = grangercausalitytests(subset, max_lag)

-->  The p-values help evaluate the significance of the causality, where lower p-values indicate a stronger evidence of causality.

In [ ]:
for lag in range(1, max_lag + 1):
    p_value = results[lag][0]['ssr_chi2test'][1]
    print(f"Lag {lag}: p-value = {p_value}")